In [ ]:
import os
import time
import requests
from dotenv import load_dotenv

load_dotenv(override=True)

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
API_URL = "https://api.groq.com/openai/v1/chat/completions"

system_prompt = """
Você é um assistente de IA muito prestativo que vai analisar um diálogo e
dizer se há a confirmação que o atendente realmente corresponde ao hotel
que o usuário deseja falar. Deve responder em: SIM, NAO ou INCONCLUSIVO.
"""

user_query = "usuário: É do hotel tal?\natendente: Não é não"

json = {
    "model": "meta-llama/llama-4-maverick-17b-128e-instruct",
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Trecho da conversa:\n\n{user_query}"}
    ],
    "temperature": 0.0,
    "response_format": {
        "type": "json_schema",
        "json_schema": {
            "name": "classify_hotel_confirmation",
            "description": "Classifica se houve confirmação de que o atendente é do hotel desejado",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                    "result": {
                        "type": "string",
                        "description": "Classificação do histórico do chat entre SIM, NAO e INCONCLUSIVO",
                        "enum": ["SIM", "NAO", "INCONCLUSIVO"]
                    }
                },
                "required": ["result"],
                "additionalProperties": False
            }
        }
    }
}


headers = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}


In [17]:
t1 = time.time()
resp = requests.post(API_URL, headers=headers, json=json)
t2 = time.time()

resp.raise_for_status()
data = resp.json()
result = data["choices"][0]["message"]["content"].strip()

print(result)
print(f"{(t2 - t1):.3f} segundos")

{
  "result": "NAO"
}
0.483 segundos
